# O Problema

Neste estudo de caso o nosso ambiente é uma lanchonete localizada na interseção entre 3 grandes supermercados da cidade. Por estar em um bom local, o fluxo de pessoas pela loja é bem intenso, composto principalmente de pessoas que estão realizando suas compras e param para um lanche rápido.

Dado este contexto, a lanchonete quer entender qual a melhor forma de montar o cardápio de salgados para maximizar o lucro líquido do dia. Ela conta com dois tipos principais de salgados, os fritos e os assados. O portfólio de fritos é composto por coxinha e pasteis de carne e de frango. Já os assados são joelho, empadinha, pão de queijo e esfirras de carne e de frango.

Cada salgado possui uma receita específica, detalhadas no documento JSON chamado *receitas.json*. No grosso modo estas receitas compartilham trigo, leite, óleo e fermento como matérias-prima. O fluxo de compra diário deste materiais pode ser feito por demanda ou de forma fixada, hoje existem estas duas possibilidades.

Vamos explorar nos próximos tópicos algumas análises e abordagens de otimização, sigamos em frente.

## Restrições

Pelo histórico da lanchonete, foi encontrado que é possível se vender em média 300 salgados fritos e 400 salgados assados por dia, então:

$ \sum_{i=1}^n x_i \leq 300 $

$ \sum_{j=1}^m x_j \leq 400 $

Em que i pertence ao conjunto de salgados fritos e j ao conjunto de salgados assados.

Esta é a restrição base do nosso estudo de caso. Porém, em algumas aplicações trabalharemos com as restrições de matérias-prima, confrontando com a não utilização das mesmas para a otimização. Neste momento, a ideia é avaliar se é possível reduzir o desperdício e o custo dos ingredientes.

In [1]:
import os
import json
import random
from enum import Enum
from typing import List, Dict, Tuple
from typing_extensions import Self

BASE_PATH = os.path.dirname(os.getcwd())
RECIPES_PATH = f"{BASE_PATH}/data/receitas.json"

In [2]:
with open(RECIPES_PATH) as file:
    recipes = json.load(file)

In [3]:
class SnackType(Enum):
    ASSADO = 1
    FRITO = 2

In [4]:
class Ingredient:
    def __init__(self, name: str, unit_cost: float,
        unit_quantity: float, buy_in_fraciont: bool
    ) -> None:
        self.name = name
        self.unit_cost = unit_cost
        self.unit_quantity = unit_quantity
        self.buy_in_fraction = buy_in_fraciont

In [20]:
class Plan:
    def __init__(self) -> None:
        self.production: Dict[str, List[int]] = {
            SnackType.FRITO.name: [],
            SnackType.ASSADO.name: []
        }
    
    def get_solution(self) -> Tuple[List[int], List[int]]:
        return [self.production[s] for s in self.production.keys()]
    
    def copy(self) -> Self:
        new_plan = Plan()
        for snack_type in self.production.keys():
            for item in self.production[snack_type]:
                new_plan.production[snack_type].append(item)
        return new_plan

In [6]:
class Recipe:
    def __init__(self, name: str, snack_type: SnackType, unit_per_batch: int,
        unit_price: float, ingredients: List[Tuple[Ingredient, float]]
    ) -> None:
        self.name = name
        self.snack_type = snack_type
        self.unit_price = unit_price
        self.ingredients = ingredients
        self.unit_per_batch = unit_per_batch

In [7]:
class Bar:
    def __init__(self, recipes: dict, max_fried: int, max_roast: int) -> None:
        self.ingredients: Dict[str, Ingredient] = dict()
        self.snacks: Dict[str, List[Recipe]] = {
            SnackType.FRITO.name: [],
            SnackType.ASSADO.name: []
        }
        self.max_production = {
            SnackType.FRITO.name: max_fried,
            SnackType.ASSADO.name: max_roast
        }
        self.__process(recipes)

    def random_plan(self) -> Plan:
        plan = Plan()
        for snack_type in self.snacks.keys():
            for _ in self.snacks[snack_type]:
                max_production = self.max_production[snack_type]
                value = random.randint(0, int(max_production * 0.2))
                current_production = sum(plan.production[snack_type])

                if current_production + value > max_production:
                    plan.production[snack_type].append(max_production - current_production)
                else:
                    plan.production[snack_type].append(value)

        return plan
    
    def near_plans(self, plan: Plan, number=50, mutation=0.1) -> List[Plan]:
        solutions: List[Plan] = []
        for _ in range(0, number):
            new_plan = plan.copy()

            for snack_type in plan.production.keys():
                while True:
                    i = random.randint(0, len(plan.production[snack_type]) - 1)
                    step = int(mutation * self.max_production[snack_type])
                    new_plan.production[snack_type][i] = new_plan.production[snack_type][i] + random.randint(-1 * step, step)

                    if sum(new_plan.production[snack_type]) <= self.max_production[snack_type]:
                        break
                    else:
                        new_plan.production[snack_type][i] = plan.production[snack_type][i]

            solutions.append(new_plan)

        return solutions
    
    def calculate_profit(self, plan: Plan) -> float:
        cost, revenue = 0, 0
        stock = dict()
        for ingredient in self.ingredients.keys():
            stock[ingredient] = 0

        for snack_type in plan.production.keys():
            for production_quantity, recipe in zip(plan.production[snack_type], self.snacks[snack_type]):
                # Faz o cálculo de quanto será preciso comprar de cada ingrediente
                for ingredient, quantity in recipe.ingredients:
                    need_quantity = quantity * production_quantity / recipe.unit_per_batch

                    # Executa a compra do material necessário
                    while stock[ingredient.name] < need_quantity:
                        if ingredient.buy_in_fraction:
                            stock[ingredient.name] = stock[ingredient.name] + need_quantity
                            cost = cost + (need_quantity / ingredient.unit_quantity) * ingredient.unit_cost
                        else:
                            stock[ingredient.name] = stock[ingredient.name] + ingredient.unit_quantity
                            cost = cost + ingredient.unit_cost

                    # Remove quantidade do estoque
                    stock[ingredient.name] = stock[ingredient.name] - need_quantity

                # Executa a venda dos produtos
                revenue = revenue + production_quantity * recipe.unit_price

        return revenue - cost

    def __process(self, recipes) -> None:
        # Carrega as matérias primas disponíveis para as receitas
        for product in recipes["materias_prima"]:
            name = product["nome"]
            self.ingredients[name] = Ingredient(
                name,
                product["custo_unitario"],
                product["quantidade_unitaria"],
                product["compra_fracionada"]
            )

        # Carrega as receitas de salgados por tipo
        for snack in recipes["salgados"]:
            snack_type = snack["tipo"].upper()
            self.snacks[snack_type].append(Recipe(
                snack["nome"],
                snack_type,
                snack["rendimento"],
                snack["vendido_por"],
                [(self.ingredients[ingredient["nome"]], ingredient["quantidade"]) for ingredient in snack["ingredientes"]]
            ))

In [54]:
class HillClimbing:
    def __init__(self, bar: Bar) -> None:
        self.bar = bar

    def execute(self, repeat=50, log=True, initial_solution: Plan=None) -> Tuple[float, Plan]:
        global_profit = 0
        global_plan = None

        for i in range(0, repeat):
            iteration = 1
            if initial_solution is not None:
                current_plan = initial_solution
            else:
                current_plan = self.bar.random_plan()
            current_profit = self.bar.calculate_profit(current_plan)

            while True:
                if log:
                    print(f"----Repetição {i + 1} iteração {iteration}, lucro de R$ {round(current_profit, 2)}----", end="\r")
                    
                improved = False
                for new_plan in self.bar.near_plans(current_plan, number=200):
                    new_profit = self.bar.calculate_profit(new_plan)
                    if new_profit > current_profit:
                        current_profit = new_profit
                        current_plan = new_plan
                        improved = True

                if current_profit > global_profit:
                    global_profit = current_profit
                    global_plan = current_plan

                iteration = iteration + 1
                if not improved:
                    break
        
        if log:
            print(f"\nMelhor lucro R$ {round(global_profit, 2)}")
            print(f"Plano de produção: {global_plan.get_solution()}")

        return global_profit, global_plan

In [62]:
class GeneticAlgorithm:
    def __init__(self, bar: Bar) -> None:
        self.bar = bar

    def random_population(self, number=50) -> List[Plan]:
        population: List[Plan] = []
        for _ in range(0, number):
            population.append(self.bar.random_plan())
        return population
    
    def tournament(self, population: List[Plan]) -> Plan:
        i = random.randint(0, len(population) - 1)
        j = random.randint(0, len(population) - 1)
        while i == j:
            j = random.randint(0, len(population) - 1)

        iprofit = self.bar.calculate_profit(population[i])
        jprofit = self.bar.calculate_profit(population[j])

        if iprofit > jprofit:
            return population[i]
        else:
            return population[j]
        
    def mutate(self, plan: Plan, mutate_rate=0.1) -> None:
        if random.random() < mutate_rate:
            for snack_type in plan.production.keys():
                values = plan.production[snack_type]
                i = random.randint(0, len(values) - 1)
                j = random.randint(0, len(values) - 1)
                values[i], values[j] = values[j], values[i]
        
    def crossover(self, plan1: Plan, plan2: Plan) -> Tuple[Plan, Plan]:
        child1, child2 = Plan(), Plan()

        for snack_type in plan1.production.keys():
            # Separa as partes de soluções para cruzamento
            i = random.randint(1, len(plan1.production[snack_type]) - 2)
            part1, rest1 = plan1.production[snack_type][0:i], plan1.production[snack_type][i:]
            part2, rest2 = plan2.production[snack_type][0:i], plan2.production[snack_type][i:]

            # Preenche as partes, mantendo a restrição de soma
            for value in rest2:
                if sum(part1) + value > self.bar.max_production[snack_type]:
                    part1.append(self.bar.max_production[snack_type] - sum(part1))
                else:
                    part1.append(value)

            for value in rest1:
                if sum(part2) + value > self.bar.max_production[snack_type]:
                    part2.append(self.bar.max_production[snack_type] - sum(part2))
                else:
                    part2.append(value)

            # Atribui os valores para os planos filhos
            child1.production[snack_type] = part1
            child2.production[snack_type] = part2

        return child1, child2
    
    def best_individual(self, population: List[Plan]) -> Tuple[Plan, float]:
        best_profit = 0
        best_plan = None

        for plan in population:
            profit = self.bar.calculate_profit(plan)
            if profit > best_profit:
                best_profit = profit
                best_plan = plan

        return best_plan, best_profit

    def execute(self, population_number=50, generations=100, mutate_rate=0.1, local_search: HillClimbing=None) -> None:
        population = self.random_population(population_number)

        for i in range(0, generations):
            new_population = []
            best_plan, best_profit = self.best_individual(population)
            print(f"----Geração {i + 1}, melhor lucro R$ {round(best_profit, 2)} com {best_plan.get_solution()}----", end="\r")

            while len(new_population) < len(population):
                winner1 = self.tournament(population)
                winner2 = self.tournament(population)

                if local_search is not None:
                    _, winner1 = local_search.execute(repeat=1, log=False, initial_solution=winner1)
                    _, winner2 = local_search.execute(repeat=1, log=False, initial_solution=winner2)

                child1, child2 = self.crossover(winner1, winner2)
                self.mutate(child1, mutate_rate=mutate_rate)
                self.mutate(child2, mutate_rate=mutate_rate)

                new_population.append(child1)
                new_population.append(child2)

            population = new_population

In [40]:
bar = Bar(recipes, max_fried=300, max_roast=400)
hc = HillClimbing(bar)
global_profit, global_plan = hc.execute(repeat=50)

----Repetição 50 iteração 10, lucro de R$ 3779.46----
Melhor lucro R$ 3880.17
Plano de produção: [[192, 80, 27], [7, 208, 6, 78, 101]]


In [64]:
bar = Bar(recipes, max_fried=300, max_roast=400)
hc = HillClimbing(bar)
ga = GeneticAlgorithm(bar)
ga.execute(population_number=20, generations=500, local_search=hc)